# Auto-Merging Retreival - Source Group B

In [22]:
import os

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    # Set global settings
    Settings.llm = llm
    Settings.embed_model = embed_model

    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)

    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine


In [ ]:
from llama_index.llms.openai import OpenAI 
from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./SourceGroupB/source12.pdf", "./SourceGroupB/source1.pdf", "./SourceGroupB/source2.pdf", "./SourceGroupB/source3.pdf", "./SourceGroupB/source4.pdf", "./SourceGroupB/source5.pdf", "./SourceGroupB/source6.pdf", "./SourceGroupB/source7.pdf", "./SourceGroupB/source8.pdf", "./SourceGroupB/source9.pdf", "./SourceGroupB/source10.pdf", "./SourceGroupB/source11.pdf"]
).load_data()


In [24]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5),
    save_dir="./merging_index",
)


In [25]:
from llama_index.core.response.notebook_utils import display_response

query_engine = get_automerging_query_engine(index, similarity_top_k=6)

with open('./generated_questions.txt') as file:
    for line in file:
        question = line.strip()
        print(question)
        display_response(query_engine.query(question))

Are current policing methods effective in reducing crime in minority communities?
> Merging 1 nodes into parent node.
> Parent node id: 31f1f412-5bc1-4c28-9114-cc6746428b3a.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



**`Final Response:`** Current policing methods have been subject to scrutiny regarding potential bias and racial disparities, particularly in discretionary stops like "stop and frisk." Despite efforts to address concerns about bias in policing data, there is ongoing debate about the effectiveness of these methods in reducing crime in minority communities. The discussion around crime in minority communities often involves comparisons of different data sets and population-based statistics to assess the impact of policing strategies on crime rates and disparities.

How does the use of facial recognition technology impact racial profiling?
> Merging 1 nodes into parent node.
> Parent node id: 31f1f412-5bc1-4c28-9114-cc6746428b3a.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



**`Final Response:`** Facial recognition technology can potentially impact racial profiling as it relies on algorithms that may be biased towards certain racial groups. This bias can lead to disproportionate targeting and surveillance of individuals from specific racial backgrounds, contributing to racial disparities in law enforcement practices.

What are the implications of reinstating 'broken windows' policing strategies?


...

**`Final Response:`** Underreporting hate crimes on minority communities can lead to a distorted perception of the prevalence and severity of such crimes. This can result in a lack of appropriate resources and support for affected communities, as well as a failure to address systemic issues that contribute to hate crimes. Additionally, underreporting can perpetuate a cycle of discrimination and violence against minority groups, as it may embolden perpetrators who feel they can act with impunity.

How do law enforcement agencies address allegations of racial bias in their practices?


**`Final Response:`** Law enforcement agencies address allegations of racial bias in their practices by conducting research and studies to analyze data on police interactions and use of force. They compare different data sets to assess for any potential bias and to determine if there are disparities in how different racial groups are treated. By examining various sources of information and analyzing the rates of police exposure for different racial groups, law enforcement agencies aim to understand and address any potential biases in their practices.

What measures can be taken to ensure equitable treatment of all racial groups within the criminal justice system?


**`Final Response:`** Measures that can be taken to ensure equitable treatment of all racial groups within the criminal justice system include:
1. Using appropriate benchmarks for disparity research rather than relying solely on overall population metrics.
2. Analyzing data sets uncontaminated by bias to accurately assess disparities in police interactions.
3. Implementing training programs for law enforcement to address any biases that may exist in decision-making processes.
4. Conducting regular reviews and audits of police practices to identify and rectify any disparities in treatment.
5. Promoting transparency and accountability within law enforcement agencies to build trust with all communities.
6. Encouraging diversity and inclusion within law enforcement to reflect the communities they serve.
7. Collaborating with community stakeholders to develop and implement policies that promote fairness and equity in the criminal justice system.